# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_sessions"
query = query + "(artist text,itemInSession int,length float,sessionId int,song text,PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        print(line[0],line[3],line[5],line[8],line[9])
        #print(type(line[0]),type(line[3]),type(line[5]),type(line[8]),type(line[9]))
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_sessions (artist,itemInSession,length,sessionId,song)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[0],int(line[3]),float(line[5]),int(line[8]),line[9]))

Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner 0 277.15873 961 Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)
Jimi Hendrix 1 239.82975 961 Woodstock Inprovisation
Building 429 2 300.61669 961 Majesty (LP Version)
The B-52's 0 321.54077 107 Love Shack
Die Mooskirchner 1 169.29914 107 Frisch und g'sund
Carrie Underwood 2 195.70893 107 Look At Me
No Te Va Gustar 0 246.43873 833 Ya EntendÃÂ­
The Velvet Underground / Nico 1 360.09751 833 All Tomorrow's Parties
Snoop Dogg / Jamie Foxx 2 179.35628 833 Psst!
Dixie Chicks 3 201.01179 833 There's Your Trouble
Linkin Park 5 189.28281 833 Given Up (Album Version)
Alice In Chains 0 239.35955 934 Take Her Out
Florence + The Machine 1 176.69179 926 My Boy Builds Coffins
Spoon 0 299.20608 968 The Mystery Zone
Alice In Chains 1 223.26812 917 Would?
Natalie Walker 2 191.84281 917 Colorblind
Traveling Wilburys 0 207.59465 940 End Of The Line
Cherokee 0 414.6673 960 Relax
NOFX 1 1099.78077 960 The Decline
Came

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select * FROM song_sessions WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(sessionid=338, iteminsession=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [46]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "DROP TABLE user_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + "(artist text,itemInSession int,sessionId int,userId int,song text, firstName text,lastName text, PRIMARY KEY (userId,sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
print("++++Table user_sessions created++++")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line[0],line[3],line[8],line[9],line[1])
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_sessions (artist,itemInSession,sessionId,userId,song,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[0],int(line[3]),int(line[8]),int(line[10]),line[9],line[1],line[4]))

print("++++Data inserted into user_sessions table++++")
                    

++++Table user_sessions created++++
++++Data inserted into user_sessions table++++


In [51]:
query = "select * from user_sessions WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(userid=10, sessionid=182, artist='Lonnie Gordon', firstname='Sylvie', iteminsession=3, lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [52]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_plays"
query = query + "(song text, firstName text,lastName text, PRIMARY KEY (song))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
print("++++Table song_plays created++++")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_plays (song,firstName,lastName)"
        query = query + "VALUES (%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query,(line[9],line[1],line[4]))

print("++++Data inserted into song_plays table++++")

                    

Error from server: code=2200 [Invalid query] message="unconfigured table song_plays"
++++Table song_plays created++++
++++Data inserted into song_plays table++++


In [55]:
query = "select firstName,lastName from song_plays WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [56]:
query = "DROP TABLE song_sessions"
try:
    session.execute(query)
    print("song_sessions table dropped")
except Exception as e:
    print(e)

query = "DROP TABLE user_sessions"
try:
    session.execute(query)
    print("user_sessions table dropped")
except Exception as e:
    print(e)

query = "DROP TABLE song_plays"
try:
    session.execute(query)
    print("song_plays table dropped")
except Exception as e:
    print(e)


song_sessions table dropped
user_sessions table dropped
song_plays table dropped


### Close the session and cluster connection¶

In [57]:
session.shutdown()
cluster.shutdown()